In [17]:
from pymongo import MongoClient
import pandas as pd
import seaborn as sns

In [4]:
client = MongoClient()
candidates = client.lbj.candidates

In [5]:
pipeline = [
    {"$group": {"_id": "$industry", "count": {"$sum": 1}}}
]

In [12]:
agg = candidates.aggregate(pipeline)

In [13]:
industry_dist = dict()
for doc in agg:
    industry_dist[doc["_id"]] = doc["count"]

In [ ]:
del industry_dist[None]

In [41]:
ind_series = pd.Series(industry_dist)

In [45]:
ind_dist = ind_series / ind_series.sum()

In [54]:
sum(ind_dist.sort_values(ascending=False).values[:80])

0.90693433923662214